In [1]:
# Enlarging the screen

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

print ('Enlarging the screen is done!')

# Importing libraries

import numpy
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

pd.set_option('max_colwidth', 100)

pd.options.mode.chained_assignment = None  # default='warn'

print ('Libraries were imported successfully!')

# Loading data from sql

Server = 'LAPTOP-I7NEB9V3\SQLEXPRESS'
Database = 'Geopattern'
Driver = 'ODBC Driver 17 for SQL Server'
Database_Connection = f'mssql://@{Server}/{Database}?driver={Driver}'

engine = create_engine(Database_Connection)
connection = engine.connect()

df_my_data = pd.read_sql_query (
    "select * from my_data", connection)

df_Authors = pd.read_sql_query (
    "select * from my_data_Authors", connection)

df_North_America = pd.read_sql_query (
    "select distinct my_data.EID, my_data.Author_ID, my_data.Year, my_data.Country from my_data join \
        (select distinct EID from my_data \
        except \
        (select distinct EID from my_data \
        where Country not like 'Canada' and Country not like 'United States' or Country is Null)) a \
        on my_data.EID = a.EID", connection)

df_North_America = df_North_America.merge(df_Authors, on = 'Author_ID', how = 'left')
df_North_America = df_North_America[~((df_North_America.Country_y != 'Canada') & (df_North_America.Country_y != 'United States'))]
df_North_America = df_North_America[['EID', 'Author_ID', 'Year', 'Country_x']]
df_North_America.rename(columns={'Country_x' : 'Country'}, inplace = True)

df_North_America.Year = df_North_America.Year.astype(int)

df_LDA = pd.read_csv(r'C:\Users\moham\Dropbox\QSE\Thesis\Geopattern\My data\df_LDA.csv')
df_LDA.set_index('EID', inplace = True)


print ('Loading data from sql is done!')

width_ind = 3
width_dep = 2

df_data_set = pd.DataFrame()

list_years = []

for yrs in range(21 - width_ind - width_dep):
    list_years.append(2000 + yrs)


for yr in tqdm(list_years, desc = 'Preparing the data set'):


    Ind_win_start = yr
    Ind_win_end = Ind_win_start + width_ind - 1

    dep_win_start = Ind_win_end + 1
    dep_win_end = dep_win_start + width_dep - 1

    df_ind = df_North_America[(df_North_America.Year > (Ind_win_start - 1)) & (df_North_America.Year <= Ind_win_end)]
    df_dep = df_North_America[(df_North_America.Year > (dep_win_start - 1)) & (df_North_America.Year <= dep_win_end)]

    #     df_ind.rename({'EID' : 'EIDs_ind'}, axis = 1, inplace = True)
    #     df_dep.rename({'EID' : 'EIDs_dep'}, axis = 1, inplace = True)


    # Targets.................................................................................................. 

    list_authors_dep = df_dep.Author_ID.unique().tolist()
    list_authors_ind = df_ind.Author_ID.unique().tolist()
    
    list_authors = list(set(list_authors_dep).intersection(list_authors_ind))
    list_authors.sort()

    df_authors_dep = pd.DataFrame(data = list_authors, columns = ['Author_ID'])
    df_authors_dep.insert(1,'EIDs','')

    df_authors_dep = df_authors_dep.merge(df_Authors, on = 'Author_ID', how = 'left')

    authors_dep = []
    for i in range (df_authors_dep.shape[0]):
        for j in range (df_authors_dep.shape[0]):
            if j > i:
                authors_dep.append((df_authors_dep['Author_ID'][i] + df_authors_dep['Author_ID'][j], df_authors_dep['Author_ID'][i], df_authors_dep['Author_ID'][j]))

    df_data_set_dep = pd.DataFrame(data = authors_dep, columns=['Author_1_2', 'Author_1', 'Author_2'])

    df_data_set_dep = df_data_set_dep.set_index(['Author_1_2'])

    df_data_set_dep.insert(2,'number_of_collaborations',0)
    df_data_set_dep.insert(3,'collaboration_binary',0)

    df_dep.reset_index(inplace = True)

    df_authors_dep['EIDs'] = ''
    for i in range (df_authors_dep.shape[0]):
        for j in range (df_dep.shape[0]):
            if df_authors_dep['Author_ID'][i] == df_dep['Author_ID'][j]:
                df_authors_dep['EIDs'][i] = str (df_authors_dep['EIDs'][i]) + ';' + str (df_dep['EID'][j])

    res = []

    for i in range (df_authors_dep.shape[0]):
        l = df_authors_dep['EIDs'][i].split(';')
        l.remove('')
        res.append((df_authors_dep['Author_ID'][i],set(l)))


    collab_matrix = np.zeros((df_authors_dep.shape[0],df_authors_dep.shape[0]))

    for i in range (len(res)):
        for j in range (len(res)):
            collab_matrix[i,j] = len(res[i][1].intersection(res[j][1]))

    collab_list = []
    for i in range (collab_matrix.shape[0]):
        for j in range (collab_matrix.shape[0]):
            if j > i:
                if collab_matrix[i,j] != 0:
                    collab_list.append((df_authors_dep['Author_ID'][i] + df_authors_dep['Author_ID'][j], df_authors_dep['Author_ID'][i], df_authors_dep['Author_ID'][j], collab_matrix[i,j]))

    df_collab = pd.DataFrame(data = collab_list, columns=['Author_1_2', 'Author_1', 'Author_2', 'number_of_collaborations'])

    df_collab = df_collab.set_index(['Author_1_2'])


    for i in df_data_set_dep.index:
        try:
            df_data_set_dep.loc[i,'number_of_collaborations'] = df_collab.loc[i,'number_of_collaborations']
        except:
            df_data_set_dep.loc[i,'number_of_collaborations'] = 0


    df_data_set_dep.collaboration_binary = df_data_set_dep.number_of_collaborations.map(lambda x: 1 if x > 0 else 0)


    # Features............................................................................................

    df_authors_ind = pd.DataFrame(data = list_authors, columns = ['Author_ID'])
    df_authors_ind.insert(1,'EIDs','')
    df_authors_ind.insert(2,'partners',0)
    df_authors_ind.insert(3,'topic_1',0)
    df_authors_ind.insert(4,'topic_2',0)
    df_authors_ind.insert(5,'topic_3',0)
    df_authors_ind.insert(6,'topic_4',0)
    df_authors_ind.insert(7,'topic_5',0)
    df_authors_ind.insert(8,'topic_6',0)
    df_authors_ind.insert(9,'topic_7',0)
    df_authors_ind.insert(10,'topic_8',0)
    df_authors_ind.insert(11,'topic_9',0)


    df_authors_ind = df_authors_ind.merge(df_Authors, on = 'Author_ID', how = 'left')

    df_authors_ind.reset_index(inplace = True)
    df_ind.reset_index(inplace = True)

    df_authors_ind['EIDs'] = ''

    for i in range (df_authors_ind.shape[0]):
        for j in range (df_ind.shape[0]):
            if df_authors_ind['Author_ID'][i] == df_ind['Author_ID'][j]:
                df_authors_ind['EIDs'][i] = str (df_authors_ind['EIDs'][i]) + ';' + str (df_ind['EID'][j])
                df_authors_ind.loc[i, 'topic_1'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_1']
                df_authors_ind.loc[i, 'topic_2'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_2']
                df_authors_ind.loc[i, 'topic_3'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_3']
                df_authors_ind.loc[i, 'topic_4'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_4']
                df_authors_ind.loc[i, 'topic_5'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_5']
                df_authors_ind.loc[i, 'topic_6'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_6']
                df_authors_ind.loc[i, 'topic_7'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_7']
                df_authors_ind.loc[i, 'topic_8'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_8']
                df_authors_ind.loc[i, 'topic_9'] += df_LDA.loc[str (df_ind['EID'][j]), 'topic_9']

    for i in range (df_authors_ind.shape[0]):
        summ = df_authors_ind.loc[i,'topic_1'] + df_authors_ind.loc[i,'topic_2'] + df_authors_ind.loc[i,'topic_3'] + df_authors_ind.loc[i,'topic_4'] + df_authors_ind.loc[i,'topic_5'] + df_authors_ind.loc[i,'topic_6'] + df_authors_ind.loc[i,'topic_7'] + df_authors_ind.loc[i,'topic_8'] + df_authors_ind.loc[i,'topic_9']
        if summ > 0:
            df_authors_ind.loc[i,'topic_1'] = df_authors_ind.loc[i,'topic_1']/summ
            df_authors_ind.loc[i,'topic_2'] = df_authors_ind.loc[i,'topic_2']/summ
            df_authors_ind.loc[i,'topic_3'] = df_authors_ind.loc[i,'topic_3']/summ
            df_authors_ind.loc[i,'topic_4'] = df_authors_ind.loc[i,'topic_4']/summ
            df_authors_ind.loc[i,'topic_5'] = df_authors_ind.loc[i,'topic_5']/summ
            df_authors_ind.loc[i,'topic_6'] = df_authors_ind.loc[i,'topic_6']/summ
            df_authors_ind.loc[i,'topic_7'] = df_authors_ind.loc[i,'topic_7']/summ
            df_authors_ind.loc[i,'topic_8'] = df_authors_ind.loc[i,'topic_8']/summ
            df_authors_ind.loc[i,'topic_9'] = df_authors_ind.loc[i,'topic_9']/summ

    res = []

    for i in range (df_authors_ind.shape[0]):
        l = df_authors_ind['EIDs'][i].split(';')
        l.remove('')
        res.append((df_authors_ind['Author_ID'][i],set(l)))


    collab_matrix = np.zeros((df_authors_ind.shape[0],df_authors_ind.shape[0]))

    for i in range (len(res)):
        for j in range (len(res)):
            collab_matrix[i,j] = len(res[i][1].intersection(res[j][1]))

    collab_list = []
    for i in range (collab_matrix.shape[0]):
        for j in range (collab_matrix.shape[0]):
            if j > i:
                if collab_matrix[i,j] != 0:
                    collab_list.append((df_authors_ind['Author_ID'][i] + df_authors_ind['Author_ID'][j], df_authors_ind['Author_ID'][i], df_authors_ind['Author_ID'][j], collab_matrix[i,j]))

    df_collab = pd.DataFrame(data = collab_list, columns=['Author_1_2', 'Author_1', 'Author_2', 'number_of_collaborations'])


    df_collab = df_collab.set_index(['Author_1_2'])

    df_authors_ind ['partners'] = ''
    for i in range (collab_matrix.shape[0]):
        for j in range (collab_matrix.shape[0]):
            if i != j:
                if collab_matrix[i,j] != 0:
                    df_authors_ind ['partners'][i] = str (df_authors_ind ['partners'][i]) + ';' + str (df_authors_ind ['Author_ID'][j])


    df_data_set_ind = pd.DataFrame(data = authors_dep, columns=['Author_1_2', 'Author_1', 'Author_2'])

    df_data_set_ind = df_data_set_ind.set_index(['Author_1_2'])

    df_data_set_ind.insert(2,'TENB',0)
    df_data_set_ind.insert(3,'Cog_Dist', '')
    df_data_set_ind.insert(4,'Geo_Dist',0)
    df_data_set_ind.insert(5,'Prov_Border',0)
    df_data_set_ind.insert(6,'NotContig',0)

    # TENB

    res_p = []

    for i in range (df_authors_ind.shape[0]):
        l = df_authors_ind['partners'][i].split(';')
        l.remove('')
        res_p.append((df_authors_ind['Author_ID'][i],set(l)))

    common_partners_matrix = np.zeros((df_authors_ind.shape[0],df_authors_ind.shape[0]))

    for i in range (len(res_p)):
        for j in range (len(res_p)):
            common_partners_matrix[i,j] = len(res_p[i][1].intersection(res_p[j][1]))


    df_common_partners = pd.DataFrame([])
    df_common_partners.insert(0,'Author_1_2','')
    df_common_partners.insert(1,'Author_1','')
    df_common_partners.insert(2,'Author_2','')
    df_common_partners.insert(3,'Common_partners',{})
    df_common_partners.insert(4,'TENB',float)


    for i in range (common_partners_matrix.shape[0]):
        for j in range (common_partners_matrix.shape[0]):
            if j > i:
                if common_partners_matrix[i,j] != 0:
                    df_common_partners = df_common_partners.append({'Author_1_2': df_authors_ind['Author_ID'][i] + df_authors_ind['Author_ID'][j],'Author_1': df_authors_ind['Author_ID'][i], 'Author_2': df_authors_ind['Author_ID'][j], 'Common_partners': res_p[i][1].intersection(res_p[j][1])}, ignore_index = True)


    number_of_articles = []
    for i in range (collab_matrix.shape[0]):
        number_of_articles.append((df_authors_ind['Author_ID'][i], collab_matrix[i,i]))

    df_number_of_articles = pd.DataFrame(data = number_of_articles, columns=['Author', 'number_of_articles'])

    df_number_of_articles_ = df_number_of_articles.set_index(['Author'])

    df_collab_ = df_collab.set_index(['Author_1' , 'Author_2'])


    for i in range (df_common_partners.shape[0]):
        n = len(df_common_partners['Common_partners'][i])
        list_ENB = []
        for j in range (n):
            common_partner = list (df_common_partners['Common_partners'][i])[j]
            d = df_number_of_articles_.loc[common_partner,'number_of_articles']
            num_article_common_partner = int(d)
            Auth_1 = df_common_partners['Author_1'][i]
            try:
                x = df_collab_.loc[(Auth_1,common_partner),'number_of_collaborations']
            except KeyError:
                x = 0
            if x != 0:
                num_collab_Auth_1 = x
            else:
                num_collab_Auth_1 = df_collab_.loc[(common_partner,Auth_1),'number_of_collaborations']
            Auth_2 = df_common_partners['Author_2'][i]
            try:
                y = df_collab_.loc[(Auth_2,common_partner),'number_of_collaborations']
            except KeyError:
                y = 0
            if y != 0:
                num_collab_Auth_2 = y
            else:
                num_collab_Auth_2 = df_collab_.loc[(common_partner,Auth_2),'number_of_collaborations']
            ENB = ((int(num_collab_Auth_1)) * (int(num_collab_Auth_2))) / num_article_common_partner
            list_ENB.append(ENB)
            TENB = sum(list_ENB)
        df_common_partners['TENB'][i] = TENB

    df_common_partners = df_common_partners.set_index(['Author_1_2'])

    for i in df_common_partners.index:
        df_data_set_ind.loc[i,'TENB'] = df_common_partners.loc[i,'TENB']

    df_authors_ind = df_authors_ind.set_index('Author_ID')


    #Cog_Dist

    for i in df_data_set_ind.index:
        a1 = df_authors_ind.loc[df_data_set_ind.loc[i, 'Author_1'], ['topic_1', 'topic_2', 'topic_3', 'topic_4','topic_5', 'topic_6','topic_7', 'topic_8','topic_9']]
        a1=a1.tolist()
        a2 = df_authors_ind.loc[df_data_set_ind.loc[i, 'Author_2'], ['topic_1', 'topic_2', 'topic_3', 'topic_4','topic_5', 'topic_6','topic_7', 'topic_8','topic_9']]
        a2=a2.tolist()
        if a1 != [0,0,0,0,0,0,0,0,0] and a2 != [0,0,0,0,0,0,0,0,0]:
            cor = numpy.corrcoef(a1, a2)
            df_data_set_ind.loc[i, 'Cog_Dist'] = 1 - cor[0][1]

    # Geo_Dist

    import math

    def Geo_Distance (lat_1,lon_1,lat_2,lon_2):
        R = 6373.0
        lat1 = math.radians(lat_1)
        lon1 = math.radians(lon_1)
        lat2 = math.radians(lat_2)
        lon2 = math.radians(lon_2)
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = R * c
        return distance

    dist = []

    for i in df_data_set_ind.index:
        author_1 = df_data_set_ind.Author_1[i]
        author_2 = df_data_set_ind.Author_2[i]
        auth_1_lat = df_authors_ind.Latitude[author_1]
        auth_1_lng = df_authors_ind.Longitude[author_1]
        auth_2_lat = df_authors_ind.Latitude[author_2]
        auth_2_lng = df_authors_ind.Longitude[author_2]

        dist.append((author_1, author_2, Geo_Distance(auth_1_lat,auth_1_lng,auth_2_lat,auth_2_lng)))

    df_geo_dist = pd.DataFrame(data = dist, columns=['Author_1', 'Author_2', 'GeoDist'])

    df_geo_dist.insert(3,'Author_1_2','')

    df_geo_dist.Author_1_2 = df_geo_dist.Author_1 + df_geo_dist.Author_2

    df_geo_dist.set_index('Author_1_2', inplace = True)

    df_data_set_ind.Geo_Dist = df_geo_dist.GeoDist

    df_temp = pd.merge(df_data_set_ind, df_authors_ind, left_on = 'Author_1', right_index = True, how = 'left')
    df_temp.rename({'Province_code':'Province_code_1'}, axis = 1, inplace = True)
    df_temp.drop(['EIDs','partners', 'Aff_ID', 'Latitude', 'Longitude'], axis = 1, inplace = True)
    df_temp2 = pd.merge(df_temp, df_authors_ind, left_on = 'Author_2', right_index = True, how = 'left')
    df_temp2.drop(['EIDs','partners', 'Aff_ID', 'Latitude', 'Longitude'], axis = 1, inplace = True)
    df_temp2.rename({'Province_code':'Province_code_2'}, axis = 1, inplace = True)

    def comparison_(x, y):
        if x == y:
            return 0
        else:
            return 1

    df_temp2.Prov_Border = df_temp2.apply(lambda x: comparison_(x.Province_code_1, x.Province_code_2), axis = 1)

    df_data_set_ind.Prov_Border = df_temp2.Prov_Border

    list_prov = list(df_authors_ind.Province.unique())

    dic_contig = {'Nova Scotia':['New Brunswick'],
                 'New Brunswick':['Nova Scotia','Quebec', 'Maine'],
                 'Newfoundland and Labrador':['Quebec'],
                 'Quebec':['New Brunswick', 'Newfoundland and Labrador', 'Ontario', 'Maine', 'New York', 'Vermont', 'New Hampshire'],
                 'Ontario':['Quebec', 'Manitoba', 'Michigan', 'Minnesota', 'New York', 'Ohio', 'Pennsylvania'],
                 'Manitoba':['Ontario','Saskatchewan', 'Minnesota', 'North Dakota'],
                 'Saskatchewan':['Manitoba','Alberta', 'Montana', 'North Dakota'],
                 'Alberta':['Saskatchewan','British Columbia', 'Montana'],
                 'British Columbia':['Alberta', 'Alaska', 'Montana', 'Washington', 'Idaho'],
                 'Alabama': ['Florida', ' Georgia', ' Mississippi', ' Tennessee'],
                 'Alaska': ['British Columbia'],
                 'Arizona': ['California', ' Colorado', ' Nevada', ' New Mexico', ' Utah'],
                 'Arkansas': ['Louisiana',' Mississippi',' Missouri',' Oklahoma',' Tennessee',' Texas'],
                 'California': ['Arizona', ' Nevada', ' Oregon'],
                 'Colorado': ['Arizona',' Kansas',' Nebraska',' New Mexico',' Oklahoma',' Utah',' Wyoming'],
                 'Connecticut': ['Massachusetts', ' New York', ' Rhode Island'],
                 'Delaware': ['Maryland', ' New Jersey', ' Pennsylvania'],
                 'Florida': ['Alabama', ' Georgia'],
                 'Georgia': ['Alabama',' Florida',' North Carolina',' South Carolina',' Tennessee'],
                 'Hawaii': [''],
                 'Idaho': ['Montana',' Nevada',' Oregon',' Utah',' Washington',' Wyoming', 'British Columbia'],
                 'Illinois': ['Indiana',' Iowa',' Michigan',' Kentucky',' Missouri',' Wisconsin'],
                 'Indiana': ['Illinois', ' Kentucky', ' Michigan', ' Ohio'],
                 'Iowa': ['Illinois',' Minnesota',' Missouri',' Nebraska',' South Dakota',' Wisconsin'],
                 'Kansas': ['Colorado', ' Missouri', ' Nebraska', ' Oklahoma'],
                 'Kentucky': ['Illinois',' Indiana',' Missouri',' Ohio',' Tennessee',' Virginia',' West Virginia'],
                 'Louisiana': ['Arkansas', ' Mississippi', ' Texas'],
                 'Maine': ['New Hampshire', 'Quebec', 'New Brunswick'],
                 'Maryland': ['Delaware', ' Pennsylvania', ' Virginia', ' West Virginia', 'District of Columbia'],
                 'Massachusetts': ['Connecticut',' New Hampshire',' New York',' Rhode Island',' Vermont'],
                 'Michigan': ['Illinois',' Indiana',' Minnesota',' Ohio',' Wisconsin', 'Ontario'],
                 'Minnesota': ['Iowa',' Michigan',' North Dakota',' South Dakota',' Wisconsin', 'Manitoba', 'Ontario'],
                 'Mississippi': ['Alabama', ' Arkanssas', ' Louisiana', ' Tennessee'],
                 'Missouri': ['Arkansas',' Illinois',' Iowa',' Kansas',' Kentucky',' Nebraska',' Oklahoma',' Tennessee'],
                 'Montana': ['Idaho', ' North Dakota', ' South Dakota', ' Wyoming', 'British Columbia', 'Alberta', 'Saskatchewan'],
                 'Nebraska': ['Colorado',' Iowa',' Kansas',' Missouri',' South Dakota',' Wyoming'],
                 'Nevada': ['Arizona', ' California', ' Idaho', ' Oregon', ' Utah'],
                 'New Hampshire': ['Maine', ' Massachusetts', ' Vermont', 'Quebec'],
                 'New Jersey': ['Delaware', ' New York', ' Pennsylvania'],
                 'New Mexico': ['Arizona', ' Colorado', ' Oklahoma', ' Texas', ' Utah'],
                 'New York': ['Connecticut',' Massachusetts',' New Jersey',' Pennsylvania',' Rhode Island',' Vermont', 'Ontario', 'Quebec'],
                 'North Carolina': ['Georgia', ' South Carolina', ' Tennessee', ' Virginia'],
                 'North Dakota': ['Minnesota', ' Montana', ' South Dakota', 'Saskatchewan', 'Manitoba'],
                 'Ohio': ['Indiana',' Kentucky',' Michigan',' Pennsylvania',' West Virginia', 'Ontario'],
                 'Oklahoma': ['Arkansas',' Colorado',' Kansas',' Missouri',' New Mexico',' Texas'],
                 'Oregon': ['California', ' Idaho', ' Nevada', ' Washington'],
                 'Pennsylvania': ['Delaware',' Maryland',' New Jersey',' New York',' Ohio',' West Virginia', 'Ontario'],
                 'Rhode Island': ['Connecticut', ' Massachusetts', ' New York'],
                 'South Carolina': ['Georgia', ' North Carolina'],
                 'South Dakota': ['Iowa',' Minnesota',' Montana',' Nebraska',' North Dakota',' Wyoming'],
                 'Tennessee': ['Alabama',' Arkansas',' Georgia',' Kentucky',' Mississippi',' Missouri',' North Carolina',' Virginia'],
                 'Texas': ['Arkansas', ' Louisiana', ' New Mexico', ' Oklahoma'],
                 'Utah': ['Arizona',' Colorado',' Idaho',' Nevada',' New Mexico',' Wyoming'],
                 'Vermont': ['Massachusetts', ' New Hampshire', ' New York', 'Quebec'],
                 'Virginia': ['Kentucky',' Maryland',' North Carolina',' Tennessee',' West Virginia', 'District of Columbia'],
                 'Washington': ['Idaho', ' Oregon', 'British Columbia'],
                 'West Virginia': ['Kentucky',' Maryland',' Ohio',' Pennsylvania',' Virginia'],
                 'Wisconsin': ['Illinois', ' Iowa', ' Michigan', ' Minnesota'],
                 'Wyoming': ['Colorado',' Idaho',' Montana',' Nebraska',' South Dakota',' Utah'],
                 'District of Columbia': ['Maryland', 'Virginia']}

    df_temp = pd.merge(df_data_set_ind, df_authors_ind, left_on = 'Author_1', right_index = True, how = 'left')
    df_temp.rename({'Province':'Province_1'}, axis = 1, inplace = True)
    df_temp.drop(['EIDs','partners', 'Aff_ID', 'Latitude', 'Longitude'], axis = 1, inplace = True)
    df_temp2 = pd.merge(df_temp, df_authors_ind, left_on = 'Author_2', right_index = True, how = 'left')
    df_temp2.drop(['EIDs','partners', 'Aff_ID', 'Latitude', 'Longitude'], axis = 1, inplace = True)
    df_temp2.rename({'Province':'Province_2'}, axis = 1, inplace = True)

    def contiguity_ (x, y):
        c = dic_contig[x]
        if c.count(y) == 1:
            return 0
        else:
            return 1

    df_temp2.NotContig = df_temp2.apply(lambda x: contiguity_(x.Province_1, x.Province_2), axis = 1)

    df_data_set_ind.NotContig = df_temp2.NotContig

        # Province dummies

    one_hot = pd.get_dummies(df_temp2[['Province_1','Province_2']])
    df_data_set_ind = df_data_set_ind.join(one_hot)

    df_data_set_ = df_data_set_dep.merge(df_data_set_ind, right_index = True, left_index = True, how = 'left')
    df_data_set = pd.concat([df_data_set,df_data_set_])

    
df_data_set.reset_index(inplace = True)
df_data_set.drop(['Author_1_y', 'Author_2_y'], axis = 1, inplace = True)
df_data_set.rename({'Author_1_x' : 'Author_1'}, axis = 1, inplace = True)
df_data_set.rename({'Author_2_x' : 'Author_2'}, axis = 1, inplace = True)
df_data_set['Log_Geo_Dist'] = df_data_set.Geo_Dist.apply(lambda x :math.log1p(x))
df_data_set.fillna(0, inplace = True)

df_data_set.insert(123,'Top_regions',0)

def Top_regions (reg1, reg2):
    if reg1 == 1 and reg2 == 1:
        return 1
        
df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x.Province_1_California, x.Province_2_California), axis = 1)
df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x.Province_1_Pennsylvania, x.Province_2_Pennsylvania), axis = 1)
df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x['Province_1_New York'], x['Province_2_New York']), axis = 1)

df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x.Province_1_California, x.Province_2_Pennsylvania), axis = 1)
df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x.Province_1_Pennsylvania, x.Province_2_California), axis = 1)

df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x.Province_1_California, x['Province_2_New York']), axis = 1)
df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x['Province_1_New York'], x.Province_2_California), axis = 1)

df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x.Province_1_Pennsylvania, x['Province_2_New York']), axis = 1)
df_data_set.Top_regions = df_data_set.apply(lambda x: Top_regions(x['Province_1_New York'], x.Province_2_Pennsylvania), axis = 1)

df_data_set.Top_regions.fillna(0, inplace = True)

Enlarging the screen is done!
Libraries were imported successfully!
Loading data from sql is done!


Preparing the data set:   0%|          | 0/16 [00:00<?, ?it/s]

In [11]:
df_data_set

,Author_1_2,Author_1,Author_2,number_of_collaborations,collaboration_binary,TENB,Cog_Dist,Geo_Dist,Prov_Border,NotContig,...,Province_1_New Hampshire,Province_1_Vermont,Province_2_Montana,Province_2_Nebraska,Province_2_Nevada,Province_2_New Brunswick,Province_2_New Hampshire,Province_2_Vermont,Log_Geo_Dist,Top_regions
0,67014571307003311854,6701457130,7003311854,0.0,0,0.0,0.625712,1019.035519,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.927593,0.0
1,67014571307003877945,6701457130,7003877945,0.0,0,0.0,0.080319,2314.353416,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.747318,0.0
2,67014571307003958245,6701457130,7003958245,0.0,0,0.0,0.385086,2020.144028,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.611419,0.0
3,67014571307004928552,6701457130,7004928552,0.0,0,0.0,0.175454,2073.925352,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.637680,0.0
4,67014571307102873710,6701457130,7102873710,0.0,0,0.0,0.744790,1018.880122,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.927440,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253070,92444882009741637700,9244488200,9741637700,0.0,0,0.0,1.269401,580.538256,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.365677,0.0
253071,92444882009841429500,9244488200,9841429500,0.0,0,0.0,0.994678,4074.869143,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.312839,0.0
253072,97389387009741637700,9738938700,9741637700,0.0,0,0.0,1.431851,2571.835938,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.852764,0.0
253073,97389387009841429500,9738938700,9841429500,0.0,0,0.0,1.135918,1512.778246,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.322364,0.0


In [12]:
df_data_set.to_csv(r'C:\Users\moham\Dropbox\QSE\Thesis\Geopattern\My data\df_data_set_North_America.csv')